In [82]:
import pandas as pd
import numpy as np

In [124]:
data = pd.read_csv('total_out_tf.csv')
data.head()

def extraire_chiffres(texte):
    return ''.join(filter(str.isdigit, texte))


def data_wrangling(data) :

    # Remove the empty data
    data = data.dropna(axis=0)
    
    # convert rating to float
    new_rating = []
    for rat in data["Rating"].tolist() :
        val = str(rat)[0:4].replace(',', '.')
        new_rating.append(round(float(val), 2))

    data['new_rating'] = new_rating
    data = data.drop(columns=['Rating'])

    # Replace '€' with an empty string and remove spaces
    data['old_price'] = data['old_price'].str.replace('€', '')
    data['old_price'] = data['old_price'].str.strip()
    data['old_price'] = data['old_price'].str.replace('\u202f', '')
    data['old_price'] = data['old_price'].astype(float)


    # Replace '€' with an empty string and remove spaces
    data['Prix'] = data['Prix'].str.replace('€', '')
    data['Prix'] = data['Prix'].str.strip()
    data['Prix'] = data['Prix'].str.replace('\u202f', '')
    data['Prix'] = data['Prix'].astype(float)
    data.rename(columns={'Prix': 'price'}, inplace=True)

    # convert period_start and periode_end to date
    data['period_start'] = pd.to_datetime(data['period_start'])
    data['period_end'] = pd.to_datetime(data['period_end'])

    # pays netoyage
    data.drop(data[data['pays'] == "Beja"].index, inplace = True)
    data.drop(data[data['pays'] == "Santa Vittoria d'Alba"].index, inplace = True)

    # voyageur netoyage
    data.loc[data["voyageurs"] == "1 lit simple", "voyageurs"] = '1 voyageurs'
    data.loc[data["voyageurs"] == "1 lit double", "voyageurs"] = '2 voyageurs'
    data.loc[data["voyageurs"] == "1 lit queen size", "voyageurs"] = '2 voyageurs'
    data.loc[data["voyageurs"] == "1 lit", "voyageurs"] = '1 voyageurs'
    data.loc[data["voyageurs"] == "2 lits simples", "voyageurs"] = '2 voyageurs'
    data.loc[data["voyageurs"] == "2 lits", "voyageurs"] = '2 voyageurs'
    data.loc[data["voyageurs"] == "3 lits", "voyageurs"] = '3 voyageurs'
    data.loc[data["voyageurs"] == "1 petit lit deux places", "voyageurs"] = '2 voyageurs'
    data.loc[data["voyageurs"] == "4 lits simples", "voyageurs"] = '4 voyageurs'
    data.loc[data["voyageurs"] == "1 voyageurs", "voyageurs"] = '1 voyageur'

    data.drop(data[data['voyageurs'] == "1 chambre"].index, inplace = True)
    data.drop(data[data['voyageurs'] == "2 chambres"].index, inplace = True)
    data.drop(data[data['voyageurs'] == "3 chambres"].index, inplace = True)
    data.drop(data[data['voyageurs'] == "4 chambres"].index, inplace = True)
    data.drop(data[data['voyageurs'] == "1 petit lit deux places"].index, inplace = True)
    data['voyageurs'] = data['voyageurs'].apply(extraire_chiffres)
    data['voyageurs'] = data['voyageurs'].astype(int)
    
    # Bed column management
    data['bed'] = data['bed'].str.replace('· ', '')
    data = data[data["bed"].str.contains("lit") == True]
    data['bed'] = data['bed'].apply(extraire_chiffres)
    data['bed'] = data['bed'].astype(int)
    
    # bathroom
    data['bathroom'] = data['bathroom'].str.replace('· ', '')
    data = data[data["bathroom"].str.contains("Demi") == False]
    
    list = data.bathroom.value_counts()
    data.drop(data[data['bathroom'] == str(list.iloc[[4]].index[0])].index, inplace = True)
    data.drop(data[data['bathroom'] == str(list.iloc[[-1]].index[0])].index, inplace = True)
    data.drop(data[data['bathroom'] == str(list.iloc[[-2]].index[0])].index, inplace = True)
    data.drop(data[data['bathroom'] == str(list.iloc[[-3]].index[0])].index, inplace = True)
    data.loc[data["bathroom"] == str(list.iloc[[3]].index[0]), "bathroom"] = str(list.iloc[[0]].index[0])
    data.loc[data["bathroom"] == str(list.iloc[[1]].index[0]), "bathroom"] = str(list.iloc[[2]].index[0])
    data['bathroom'] = data['bathroom'].apply(extraire_chiffres)
    data['bathroom'] = data['bathroom'].astype(int)
        
    # room
    data['rooms'] = data['rooms'].str.replace('· ', '')
    data = data[data["rooms"].str.contains("chambre") == True]
    data['rooms'] = data['rooms'].apply(extraire_chiffres)
    data['rooms'] = data['rooms'].astype(int)
    
    return data


In [126]:
# Replace '€' with an empty string and remove spaces
our_data = data_wrangling(data)
our_data

/var/folders/_y/9bzh_k0d3h15227_glcddy0m0000gn/T/ipykernel_26638/1637105900.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['new_rating'] = new_rating
/var/folders/_y/9bzh_k0d3h15227_glcddy0m0000gn/T/ipykernel_26638/1637105900.py:37: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['period_start'] = pd.to_datetime(data['period_start'])
/var/folders/_y/9bzh_k0d3h15227_glcddy0m0000gn/T/ipykernel_26638/1637105900.py:38: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consist

,price,old_price,Type,pays,region,voyageurs,rooms,bed,bathroom,period_start,period_end,new_rating
0,2920.0,6084.0,cottage,Espagne,Madrid,9,3,8,2,2024-07-31,2024-08-31,4.97
1,1500.0,1705.0,hébergement,Espagne,Trevélez,2,1,1,1,2024-07-31,2024-08-31,4.89
2,1968.0,3280.0,loft,Espagne,Madrid,2,1,1,1,2024-07-31,2024-08-31,4.81
3,2353.0,2870.0,loft,Espagne,Séville,4,2,3,1,2024-07-31,2024-08-31,4.96
4,2129.0,2505.0,appartement,Espagne,Madrid,2,1,1,1,2024-07-31,2024-08-31,4.83
...,...,...,...,...,...,...,...,...,...,...,...,...
1180,2635.0,3100.0,appartement en résidence,Portugal,Cascais,2,1,1,1,2024-07-31,2024-08-31,4.71
1181,2976.0,3720.0,hébergement,Portugal,Estoril,3,2,2,1,2024-07-31,2024-08-31,5.00
1182,3367.0,5520.0,appartement,Portugal,Ericeira,4,2,3,1,2024-07-31,2024-08-31,4.88
1186,3713.0,7425.0,appartement,Portugal,Vila Nova de Gaia,4,1,2,1,2024-07-31,2024-08-31,4.43


In [127]:
our_data.to_csv('total_out_clean.csv')